In [ ]:
#  pip install langchain
#  pip install openai
#  pip install tiktoken
#  pip install faiss-cpu

In [ ]:
pip install PyPDF2

In [ ]:
pip install -U langchain-community

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
pdfreader = PdfReader("budget_speech.pdf")

In [ ]:
from typing_extensions import Concatenate
raw_text = ""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text+= content

In [ ]:
raw_text

In [ ]:
text_spiltter = CharacterTextSplitter(
    separator='\n',
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,

)

In [ ]:
text=text_spiltter.split_text(raw_text)

In [ ]:
len(text)

In [ ]:
import tiktoken
encoding = tiktoken.get_encoding('cl100k_base')

In [ ]:
def count_tokens(text:str)->int:
  return len(encoding.encode(text))

In [ ]:
text_spiltter = CharacterTextSplitter(
    separator='/n',
    chunk_size=800,
    chunk_overlap=200,
    length_function=count_tokens
)

In [ ]:
for i, page in enumerate(pdfreader.pages):
  text=page.extract_text()
  if text:
    print(f"Page {i+1} - Tokens: {count_tokens(text)}")
  else:
    print(f"Page {i+1} - No extractable text")

In [ ]:
print("Total tokens :",count_tokens(" ".join(text)))

In [ ]:
embedding = OpenAIEmbeddings()

In [ ]:
doc_search = FAISS.from_texts(text,embedding)

In [ ]:
doc_search

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(),chain_type='stuff')

In [ ]:
query = " "
docs = doc_search.similarity_search(query)
chain.run(input_documents=docs, questions=query)